# install lines

In [ ]:
# pip install tqdm

# pip install selenium

# pip install bs4

# pip install requests

# pip install pandas

# pip install openpyxl

# import lines

In [2]:
# 정적 크롤링
from bs4 import BeautifulSoup as bs

# request
from urllib.request import urlopen
import requests

# html parser
from urllib.parse import urljoin

# to excel and DataFrame
import os
import numpy
import pandas as pd

# selenium everytime session
# 동적 크롤링 lines
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By

# data storage
import os
import numpy
import pandas as pd

import time
import random
import openpyxl
from tqdm import tqdm

/Users/kimgkangmin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Pandas Data Frame

In [22]:
# 처음에만 실행시킬 것


data_frame = pd.DataFrame({"time" : [],
                           "URL" : [],
                           "title" : [],
                          "article" : [],
                          "reple" : []})

In [ ]:
# excel to df -  이어서 크롤링 하려면 저장 파일을 불러와서 clow

# data_frame = pd.read_excel('/Users/kimgkangmin/Desktop/code/python/clow_data_final.xlsx')

# Clowling lines

In [ ]:
driver = webdriver.Chrome()

# 에브리타임 로그인 페이지로 이동
driver.get('https://account.everytime.kr/login')

# 아이디 및 비밀번호 입력
driver.find_element(By.NAME, 'id').send_keys(' ## ') # put your id
driver.implicitly_wait(1)
driver.find_element(By.NAME, 'password').send_keys(' ## ') # put your password
driver.implicitly_wait(1)

# 로그인 버튼 클릭
driver.find_element(By.XPATH, '/html/body/div[1]/div/form/input').click()
driver.implicitly_wait(1)

In [24]:
# Clow Pages

def clow_page(bs, driver, page):

    pages = []
    # 핫 게시물 페이지 접속
    driver.get(f'https://everytime.kr/hotarticle/p/{page}')
    time.sleep(random.randrange(2,5))

    # 페이지 소스 가져오기
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # 게시물 목록 찾기
    titles = soup.select("#container > div.articles > article.list > a.article")
    
    for i in titles:
        links = i.attrs["href"]
        pages.append('https://everytime.kr' + links)

    if len(pages) == 20:
        print("게시물 20개 크롤링 완료")
    else:
        print(f"게시물 {len(pages)}개 크롤링 완료")

    return pages

In [ ]:
#-------------------------------------------------------------------------------------------------------------------
def time_clow(abc):
    time.sleep(random.randrange(2,5))
    s = abc.select_one("#container > div.articles > article.item time.large").get_text()
    s = ('2024' + ''.join(s.split('/')))[:8]
    s = pd.to_datetime(s)
    s = s.strftime("%Y/%m/%d")
    return s

#-------------------------------------------------------------------------------------------------------------------

def title_clow(abc):
    time.sleep(random.randrange(2,5))
    return abc.select_one("#container > div.articles > article h2").string

#-------------------------------------------------------------------------------------------------------------------

def article_clow(abc):
    time.sleep(random.randrange(2,5))
    return abc.select_one("#container > div.articles > article p").get_text()

#-------------------------------------------------------------------------------------------------------------------

def reple_clow(abc):
    result = []
    time.sleep(random.randrange(2,5))

    comments = abc.select("#container > div.articles > article.item > div.comments article p.large")

    for comment in comments:
        result.append(comment.text)

    result = '\n'.join(result)
    
    return result

#-------------------------------------------------------------------------------------------------------------------

def total_clow(data_frame, bs, driver, page, start_date, end_date):

    total_clow_df = pd.DataFrame({"time" : [],
                    "URL" : [],
                    "title" : [],
                    "article" : [],
                    "reple" : []})
         
    for p in tqdm(page):
        driver.get(p)  # 각 게시물로 이동

        time.sleep(random.randrange(2,5))

        html = driver.page_source
        abc = bs(html, 'html.parser')

        if time_clow(abc) > start_date.strftime('%Y/%m/%d') and time_clow(abc) < end_date.strftime('%Y/%m/%d'):

            if check_url(p, data_frame):
                
                print(f"{p}는 처음 크롤링 되는 글입니다")
                
                total_clow_df.loc[len(total_clow_df['URL'])] = [time_clow(abc) , p, title_clow(abc),
                                          article_clow(abc), reple_clow(abc)]
                
            else:
                print(f"URL {p}은 이미 존재합니다.")
        else:
            print("시간이 안 맞습니다")
    
    return total_clow_df

#-------------------------------------------------------------------------------------------------------------------

def check_url(url, data_frame):    
    if url not in data_frame['URL']:
        return True
    else: 
        return False

#-------------------------------------------------------------------------------------------------------------------

# 실행 line

In [ ]:
# 원하는 기간이 포함되는 대략적인 페이지 넘버를 확인 후 돌리는 것이 편함

start_page = 1 
end_page = 2


start_date = pd.to_datetime("20240831") # year, month, day 순으로 str 형식으로 넣어줄 것
end_date = pd.to_datetime("20241101")

for p in tqdm(range(start_page, end_page + 1)):
    pages = clow_page(bs, driver, p)

    total_clow_df = total_clow(data_frame, bs, driver, pages, start_date, end_date)
    
    data_frame = pd.concat([data_frame, total_clow_df], ignore_index=True)

  0%|          | 0/6 [00:00<?, ?it/s]

게시물 20개 크롤링 완료
https://everytime.kr/380952/v/355061574는 처음 크롤링 되는 글입니다
         time                                      URL              title  \
0  2024/09/21  https://everytime.kr/380952/v/355061574  흔적 너무 좋은 동아리인거 같다   

  article                                              reple  
0       .  회장이 존잘남임;;\n?\n진짜 개인정  ㅋㅋㅋ\n가지고싶다\n순간 미만잡\n횡령동...  
https://everytime.kr/380952/v/355024070는 처음 크롤링 되는 글입니다
         time                                      URL              title  \
0  2024/09/21  https://everytime.kr/380952/v/355061574  흔적 너무 좋은 동아리인거 같다   
1  2024/09/20  https://everytime.kr/380952/v/355024070      의외로 교수님들도 하는것   

       article                                              reple  
0            .  회장이 존잘남임;;\n?\n진짜 개인정  ㅋㅋㅋ\n가지고싶다\n순간 미만잡\n횡령동...  
1  그것은 바로...키배  개웃기네 ㅋㅋㅋㅋㅋㅋㅋ\n교수님이요?\n첫짱보고 나만 파카 생각했나\nㅋㅋㅋㅋㅋㅋㅋ...  
https://everytime.kr/386002/v/354864732는 처음 크롤링 되는 글입니다
         time                                      URL              title  \
0  2024/09/21  https://

 17%|█▋        | 1/6 [07:19<36:36, 439.40s/it]

          time                                      URL  \
0   2024/09/21  https://everytime.kr/380952/v/355061574   
1   2024/09/20  https://everytime.kr/380952/v/355024070   
2   2024/09/18  https://everytime.kr/386002/v/354864732   
3   2024/09/20  https://everytime.kr/259022/v/355034968   
4   2024/09/20  https://everytime.kr/380952/v/355028437   
5   2024/09/20  https://everytime.kr/380952/v/355014765   
6   2024/09/20  https://everytime.kr/259022/v/355001200   
7   2024/09/20  https://everytime.kr/380952/v/354982467   
8   2024/09/20  https://everytime.kr/380952/v/354978194   
9   2024/09/20  https://everytime.kr/380952/v/354962811   
10  2024/09/19  https://everytime.kr/380952/v/354936305   
11  2024/09/19  https://everytime.kr/380952/v/354923694   
12  2024/09/19  https://everytime.kr/380952/v/354923943   
13  2024/09/19  https://everytime.kr/380952/v/354929758   
14  2024/09/19  https://everytime.kr/380952/v/354922203   
15  2024/09/19  https://everytime.kr/380952/v/354912258 

 33%|███▎      | 2/6 [14:41<29:23, 440.86s/it]

...

          time                                      URL  \
0   2024/09/11  https://everytime.kr/380952/v/354233707   
1   2024/09/11  https://everytime.kr/380952/v/354230836   
2   2024/09/11  https://everytime.kr/380952/v/354201985   
3   2024/09/10  https://everytime.kr/380952/v/354183083   
4   2024/09/10  https://everytime.kr/380952/v/354162366   
5   2024/09/10  https://everytime.kr/380952/v/354150893   
6   2024/09/10  https://everytime.kr/380952/v/354145300   
7   2024/09/10  https://everytime.kr/380952/v/354142697   
8   2024/09/10  https://everytime.kr/380952/v/354132185   
9   2024/09/07  https://everytime.kr/380952/v/353851890   
10  2024/09/09  https://everytime.kr/380952/v/354075238   
11  2024/09/09  https://everytime.kr/380952/v/354065952   
12  2024/09/09  https://everytime.kr/380952/v/354054386   
13  2024/09/09  https://everytime.kr/380952/v/354019782   
14  2024/09/08  https://everytime.kr/259022/v/353923274   
15  2024/09/08  https://everytime.kr/259022/v/353914022 

 83%|████████▎ | 5/6 [35:44<07:03, 423.90s/it]

          time                                      URL  \
0   2024/09/02  https://everytime.kr/380952/v/353091747   
1   2024/09/06  https://everytime.kr/380952/v/353682353   
2   2024/09/02  https://everytime.kr/380952/v/353093610   
3   2024/09/05  https://everytime.kr/380952/v/353531527   
4   2024/09/05  https://everytime.kr/380952/v/353548456   
5   2024/09/04  https://everytime.kr/259022/v/353449869   
6   2024/09/04  https://everytime.kr/380952/v/353390456   
7   2024/09/03  https://everytime.kr/259022/v/353365796   
8   2024/09/03  https://everytime.kr/380952/v/353328115   
9   2024/09/03  https://everytime.kr/380952/v/353304354   
10  2024/09/03  https://everytime.kr/380952/v/353287335   
11  2024/09/02  https://everytime.kr/259022/v/353175358   
12  2024/09/02  https://everytime.kr/380952/v/353203616   
13  2024/09/02  https://everytime.kr/259022/v/353174686   
14  2024/09/02  https://everytime.kr/380952/v/353186633   
15  2024/09/02  https://everytime.kr/380952/v/353139945 

100%|██████████| 6/6 [40:02<00:00, 400.46s/it]

시간이 안 맞습니다
         time                                      URL  \
0  2024/09/02  https://everytime.kr/380952/v/353141047   
1  2024/09/02  https://everytime.kr/259022/v/353120693   
2  2024/09/02  https://everytime.kr/380952/v/353127806   
3  2024/09/02  https://everytime.kr/380952/v/353119360   
4  2024/09/02  https://everytime.kr/380952/v/353092326   
5  2024/09/02  https://everytime.kr/380952/v/353064893   
6  2024/09/01  https://everytime.kr/380952/v/352985745   
7  2024/09/01  https://everytime.kr/380952/v/352963420   

                         title  \
0  딥페이크, 디지털 성폭력, 어떻게 대응해야 할까?   
1         학교 주차 건의사항(핫게 보내주세요)   
2                      도서관 무슨일   
3        학교 주차장 건의사항(핫게 보내주세요)   
4                   종강할때 된거같은데   
5           요즘 내 학교생활 요약해봄.jpg   
6    선배랑 말싸움하다가 지 주제도 모른다고 하길래   
7      교수님,  家 이 한자는 무슨 한자인가요?   

                                             article  \
0  딥페이크 등 디지털 성폭력 피해를 입었을 때 대처하는 방법을 담은 카드뉴스입니다. ...   
1  자차로 다른 지역에서 통학하는 학생입니다제가 오늘 느꼈던 불편하고 개선했으면 좋

In [ ]:
# df 확인

data_frame.head()

Unnamed: 0        time                                      URL  \
0            0  2024/10/31  https://everytime.kr/380952/v/358786915   
1            1  2024/10/31  https://everytime.kr/380952/v/358776030   
2            2  2024/10/31  https://everytime.kr/380952/v/358767790   
3            3  2024/10/31  https://everytime.kr/380952/v/358768087   
4            4  2024/10/31  https://everytime.kr/380952/v/358747507   
5            5  2024/10/31  https://everytime.kr/380952/v/358735112   
6            6  2024/10/31  https://everytime.kr/380952/v/358728296   
7            7  2024/10/31  https://everytime.kr/380952/v/358727160   
8            8  2024/10/30  https://everytime.kr/259022/v/358641376   
9            9  2024/10/30  https://everytime.kr/542302/v/358707811   
10          10  2024/10/30  https://everytime.kr/380952/v/358677598   
11          11  2024/10/30  https://everytime.kr/380952/v/358695375   
12          12  2024/10/30  https://everytime.kr/380952/v/358673000   
13          13  2024/10/27  https://everytime.kr/542302/v/358388006   
14          14  2024/10/30  https://everytime.kr/380952/v/358653472   
15          15  2024/10/30  https://everytime.kr/380952/v/358658289   
16          16  2024/10/30  https://everytime.kr/380952/v/358655062   
17          17  2024/10/30  https://everytime.kr/380952/v/358650024   
18          18  2024/10/30  https://everytime.kr/380952/v/358646298   
19          19  2024/10/30  https://everytime.kr/380952/v/358636799   
20          20  2024/10/30  https://everytime.kr/380952/v/358619053   
21          21  2024/10/29  https://everytime.kr/380952/v/358603110   
22          22  2024/10/29  https://everytime.kr/380952/v/358600302   
23          23  2024/10/29  https://everytime.kr/380952/v/358573166   
24          24  2024/10/29  https://everytime.kr/380952/v/358551492   
25          25  2024/10/29  https://everytime.kr/380952/v/358553520   
26          26  2024/10/29  https://everytime.kr/380952/v/358549919   
27          27  2024/10/29  https://everytime.kr/380952/v/358547579   
28          28  2024/10/29  https://everytime.kr/380952/v/358541574   
29          29  2024/10/28  https://everytime.kr/380952/v/358515355   

                                    title  \
0          미니게이야... 이거 보여주려고 챗지피티 결제했다...   
1                       담배값 7000원 인상 뭐야??   
2                                     실시간   
3                              💀그 귀신 접니다👻   
4                             🎃경상대 귀신 출몰🎃   
5            건축공학과 21학번 김현동군의 생일을 축하해주세요.   
6                            오늘도 시험 잘치세요~   
7                  이젠 걍 ㅃㄹ 시험보고 자유가 되고 싶음   
8                              인생 망치는 지름길   
9                          ㅅㅁ교육과 왤케 잘생김…?   
10                                  미니 빌런   
11                             경영대앞 이거 뭐냐   
12                   🎼✨제 40회 음악교육과 정기연주회🍂   
13                           일사교 전선 추천해준다   
14                                진짜 개좆같다   
15                          학교 안에서 유명인 봤다   
16                      근데 시험이라는건 좀 야한거같음   
17                      내가 찍은거중 가장 아끼는 사진   
18  [반박] 대학생이면 알아야 하는 현재 나라 상황 <- 선동 그 잡채   
19                       아 ㅈ됐다 이거 어떡함? ㅅㅂ   
20                     시험 기간에 올리는 피크민 키우기   
21                         중앙도서관 1-2층 계단부   
22                도서관에서 이쁜사람 봐서 인스타 물어봤는데   
23                        자연대는 뭐하는 동네임 대체   
24               생활법률 45분이나 지각하는 걸 왜 봐주는데   
25                      가만히 있으면 반이라도 간다면서   
26                          경대생들 시험 잘 치세요   
27                                   생활법률   
28                          너 지금 학생이야 백수야   
29                 와..잠만 학교에서 이래도 되는거임???   

                                              article  \
0                   대 국립경상대학교의 이름을 빚네는 너희들이 자랑스럽다!!!!   
1   담배값 7000원 기사 링크 이거 되면 담배 어떻게 피냐…...... 경상국립대 학...   
2                                 빨간 스팅어 국제어학원앞 등장!!!   
3   친구들과 사진찍고 놀기위해 입고 왔습니다..할로윈을 즐겨서 행복합니다...민폐였다면...   
4                            할로윈을 밎아서 경상대에 귀신이 등장했어요!   
5                   건축공학과 21학번 김현동군의 생일을 진심으로 축하드립니다.   
6      

---

data_frame to img

---

In [ ]:
# excel에서 불러왔을 때 data_frame에 새로운 index가 붙어있다면 코드 실행

data_frame = data_frame.drop(columns='Unnamed: 0', axis=1)
data_frame[:5]

Time                                      URL  \
0  09/30 16:54  https://everytime.kr/380952/v/355878414   
1  09/30 17:17  https://everytime.kr/380952/v/355878414   
2  09/30 19:27  https://everytime.kr/380952/v/355878414   
3  09/30 15:45  https://everytime.kr/380952/v/355878414   
4  09/30 12:45  https://everytime.kr/380952/v/355878414   

                            title  \
0               축제때 재학생 존 만들어 주세요   
1                       모솔아다의 악순환   
2               제8회 💛GNU 토론대회💛 개최   
3           건축공학과 엄예빈의 생일을 축하해주세요   
4  국군의날 임시공휴일 지정으로 인한 바뀐 보강일 재 공지   

                                             article  \
0                                                  !   
1  1.학창시절 찐따생활로 찐따력 만땅 or 서브컬쳐에 빠져이성과의 교류 X, 주변에 ...   
2  안녕하세요 개척 학우 여러분!2024 국립대학육성사업💛GNU 토론대회💛가 개최됩니다...   
3                                          다들 축하해주세요   
4                                                ㅈㄱㄴ   

                                               reple  
0  싫은데용\n대학원생 존 만들어주세요!!!\n이거는 힘들 것 같군요.\n그러면 시장님...  
1  어떻게 하라고. 어떻게 하라고. 어떻게 하라고. 어떻게 하라고. 어떻게 하라고. 어...  
2  나와 겨뤄볼자 신청하라\n이번에는 총장상에다가 상금까지 100만원 됐네 ㄷㄷ\n저거...  
3  건공이쁜이 생축\n엄\n준\n식\n축하합니다 -농대-\n개큰축하 축하 빵빠레 🗯️❤...  
4  핫게보내고 스크럽하자^^\n공감은 안 하고 스크랩만 해야지\n혼나\n종강일 엄청 늦...

# transform df - excel

In [47]:
# data_frame to excel   

data_frame.to_excel('clow_data_final.xlsx')

In [ ]:
# excel to df

data_frame = pd.read_excel('/Users/kimgkangmin/Desktop/code/python/clow_data.xlsx')

20
